In this notebook we consider matching diagrams and see how well they distinguish between behaviours. Also, we see how well they relate to number of collisions, efficacy and deadlocks.

We start loading the necessary modules.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.spatial.distance as dist
import matplotlib as mpl

import os

from navground import core, sim

import perdiver.perdiver as perdiver
from perdiver.distances import *

plots_dir = os.path.join("plots", "matchings_PCA")
experiment_dir = "experiments"
os.makedirs(plots_dir, exist_ok=True)
os.makedirs("experiments", exist_ok=True)

Now, we run the corridor experiment with various behaviours.

In [ ]:
# Small Corridor
# length = 8
# width=1.0
# Large Corridor
width=2
length = 15.0
num_steps = 400

num_agents = 12 # 31
num_runs = 15
behaviour_list = ["ORCA", "HL", "HRVO", "Dummy", "SocialForce"]
marker_behaviour = {"ORCA": "o", "HL": "X", "HRVO": "+", "Dummy": "*", "SocialForce": "x"}
color_behaviour = {}
for i, behaviour in enumerate(behaviour_list):
    color_behaviour[behaviour] = mpl.colormaps["Set1"](i / (len(behaviour_list) +1))
for behaviour in behaviour_list:
    path = os.path.join(experiment_dir, f"matchings_PCA_{behaviour}.h5")
    yaml = f"""
    steps: {num_steps}
    time_step: 0.1
    record_pose: true
    record_twist: true
    runs: {num_runs}
    record_collisions: true
    record_deadlocks: true
    record_safety_violation: true
    record_efficacy: true
    terminated_when_idle_or_stuck: false
    scenario:
      type: Corridor
      length: {length}
      width: {width} 
      groups:
        -
          type: thymio
          number: {num_agents}
          radius: 0.35
          control_period: 0.1
          speed_tolerance: 0.02
          kinematics:
            type: 2WDiff
            wheel_axis: 0.094
            max_speed: 0.2
          behavior:
            type: {behaviour}
            optimal_speed: 
                sampler: normal
                mean: 0.16
                std_dev: 0.04
            horizon: 5.0
            safety_margin: 0.1
          state_estimation:
            type: Bounded
            range: 5.0
    """
    experiment = sim.load_experiment(yaml)
    experiment.run(keep=False, data_path=path)
    del experiment

And we load the saved runs.

In [ ]:
runs = {}
# Reload simulations
for behaviour in behaviour_list:
    path = os.path.join(experiment_dir, f"matchings_PCA_{behaviour}.h5")
    recorded_experiment = sim.RecordedExperiment(path)
    runs[behaviour] = recorded_experiment.runs

# PCA using Matching diagrams

Next, we compute the persistence matching diagrams. We do this by ranging over an initial step list. We add such persistence diagrams to obtain one for each run.

In [ ]:
weight = 1 # This is the weight for velocities
shift_step = 30 # Shift between timesteps
steps_list = list(range(100, 240, 10)) # Starting timesteps that we consider

In [ ]:
import scipy.spatial.distance as dist

diagrams_behaviour = {}
for j, behaviour in enumerate(behaviour_list):
    diagrams_run_list = []
    for i_run in range(num_runs):
        ps = np.array(runs[behaviour][i_run].poses)
        twists = np.array(runs[behaviour][i_run].twists)
        diagrams_list = []
        for idx, start_step in enumerate(steps_list):
            X = ps[start_step]
            Y = ps[start_step + shift_step]
            vel_X = twists[start_step]
            vel_Y = twists[start_step + shift_step]
            Dist_X, Dist_Y, Dist_Z = perdiver.compute_distance_matrices_timesteps_corridor(X, Y, vel_X, vel_Y, weight, length, False)
            # Dist_X = dist.squareform(dist.pdist(vel_X))
            # Dist_Y = dist.squareform(dist.pdist(vel_Y))
            match_diagram = perdiver.get_matching_diagram(Dist_X, Dist_Y)
            diagrams_list.append(match_diagram)
        # end for 
        diagrams_run_list.append(np.vstack(np.array(diagrams_list)))
    # end for
    diagrams_behaviour[f"{behaviour}"] = diagrams_run_list
    if behaviour=="ORCA":
        fig, ax = plt.subplots(figsize=(4,4))
        match_diagram = diagrams_run_list[-1]
        perdiver.plot_matching_diagram(match_diagram, ax, color="blue")
        plt.savefig("ORCA_matching_diagram.png")
# end for

Plot resulting diagrams for different behaviours. We should see notable differences.

In [ ]:
fig, ax = plt.subplots(ncols=len(behaviour_list), figsize=(4*len(behaviour_list),4))
for j, behaviour in enumerate(behaviour_list):
    for idx, start_step in enumerate(steps_list):
        match_diagram = diagrams_behaviour[behaviour][5]
        perdiver.plot_matching_diagram(match_diagram, ax[j], color="blue")
    # end for 
    ax[j].set_title(f"Sum matching diagram {behaviour}")
# end for
perdiver.same_diagram_scale(ax.ravel())
plt.savefig(os.path.join(plots_dir, f"diagrams_added_behaviours.png"))

### Persistence images

Now, compute persistence images.

In [ ]:
from gudhi import representations

npixels = 8
perim = representations.PersistenceImage(resolution=[npixels, npixels], bandwidth=0.3)
all_diagrams = []
for behaviour in behaviour_list:
    all_diagrams += diagrams_behaviour[behaviour]
perim.fit(all_diagrams)
perim_arr_dict = {}
for behaviour in behaviour_list:
    perim_arr_dict[behaviour] = perim.transform(diagrams_behaviour[behaviour])

all_perim = np.vstack([perim_arr_dict[behaviour] for behaviour in behaviour_list])

In [ ]:
all_perim_transformed = []
for i, behaviour in enumerate(behaviour_list):
    for image in perim_arr_dict[behaviour]:
        image_transformed = image.reshape((npixels,npixels))
        all_perim_transformed.append(image_transformed.ravel())

In [ ]:
fig, ax = plt.subplots(ncols=len(behaviour_list), figsize=(20,5))
for i, behaviour in enumerate(behaviour_list):
    image = perim_arr_dict[behaviour][1].reshape((npixels,npixels))
    image = image * np.array(list(range(npixels)))
    ax[i].set_title(behaviour)
    ax[i].imshow(image)

Do PCA on these images.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import minmax_scale

pca = PCA(n_components=2)
# scaled_all_perim = minmax_scale(all_perim_transformed)
# Y = pca.fit_transform(scaled_all_perim)
Y = pca.fit_transform(all_perim_transformed[:-num_runs])

Y_dict = {}
for i, behaviour in enumerate(behaviour_list):
    Y_dict[behaviour] = Y[num_runs*i:num_runs*(i+1)]
# end for
# Plot PCA
fig, ax = plt.subplots(figsize=(5,5))
for behaviour in behaviour_list:
    # if behaviour=="HRVO":
    #     continue
    ax.scatter(Y_dict[behaviour][:,0], Y_dict[behaviour][:,1], color=color_behaviour[behaviour], label=behaviour)

handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
fig.legend(by_label.values(), by_label.keys(), loc=(0.1,0),  ncol=len(behaviour_list))
plt.tight_layout()
plt.savefig(os.path.join(plots_dir, "PCA-projection.png"))

We also try with UMAP

In [ ]:
import umap

reducer = umap.UMAP()
embedding = reducer.fit_transform(all_perim_transformed)
fig, ax = plt.subplots(figsize=(5,5))
embedding_dict = {}
for i, behaviour in enumerate(behaviour_list):
    embedding_dict[behaviour] = embedding[num_runs*i:num_runs*(i+1)]
# end for
for behaviour in behaviour_list:
    # if behaviour=="HRVO":
    #     continue
    ax.scatter(embedding_dict[behaviour][:,0], embedding_dict[behaviour][:,1], color=color_behaviour[behaviour], label=behaviour)

plt.savefig(os.path.join(plots_dir, "UMAP-projection.png"))

See how this projection relates to efficacy, collisions and deadlocks.

In [ ]:
import pandas as pd

def count_deadlocks(deadlock_time, initial_time, final_time):
    is_deadlocked = np.logical_and(deadlock_time > initial_time, deadlock_time < (final_time - 5.0))
    return sum(is_deadlocked)

def extract_data(experiment, initial_step, final_step):
    collisions = []
    deadlocks = []
    efficacy = []
    sms = []
    seeds = []
    for i, run in experiment.runs.items():
        world = run.world
        initial_time, final_time = initial_step*run.time_step, final_step*run.time_step
        deadlocks.append(count_deadlocks(np.array(run.deadlocks), initial_time, final_time))
        collisions.append(np.sum(np.logical_and(
            initial_step < run.collisions[:,0], run.collisions[:,0] < final_step
        )))
        efficacy.append(np.array(run.efficacy[initial_step:final_step]).mean())

    df = pd.DataFrame({
        'deadlocks': deadlocks,
        'collisions': collisions,
        'efficacy': efficacy})
    df['safe'] = (df.collisions == 0).astype(int)
    df['fluid'] = (df.deadlocks == 0).astype(int)
    df['ok'] = ((df.deadlocks == 0) & (df.collisions == 0)).astype(int)
    return df

In [ ]:
initial_step = 0
final_step = 100
deadlocks = {}
collisions = {}
efficacy = {}
for behaviour in behaviour_list:
    path = os.path.join(experiment_dir, f"matchings_PCA_{behaviour}.h5")
    recorded_experiment = sim.RecordedExperiment(path)
    df = extract_data(recorded_experiment, initial_step, final_step)
    collisions[behaviour] =  list(df.collisions)
    deadlocks[behaviour] = list(df.deadlocks)
    efficacy[behaviour] = list(df.efficacy)

Plot classes against efficacy, collisions and deadlocks

In [ ]:
fig, ax = plt.subplots(figsize=(12,4), ncols=3)
for behaviour in behaviour_list:
    ax[0].scatter(deadlocks[behaviour], collisions[behaviour], color=color_behaviour[behaviour], label=behaviour)
    ax[0].set_xlabel("deadlocks")
    ax[0].set_ylabel("collisions")
    ax[1].scatter(efficacy[behaviour], collisions[behaviour], color=color_behaviour[behaviour], label=behaviour)
    ax[1].set_xlabel("efficacy")
    ax[1].set_ylabel("collisions")
    ax[2].scatter(efficacy[behaviour], deadlocks[behaviour], color=color_behaviour[behaviour], label=behaviour)
    ax[2].set_xlabel("efficacy")
    ax[2].set_ylabel("deadlocks")
# end for
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
fig.legend(by_label.values(), by_label.keys(), loc=(0.3,0),  ncol=len(behaviour_list))
plt.tight_layout()
plt.savefig(os.sep.join((plots_dir, "efficiency_collisions_corridor.png")))

In [ ]:
# Plot efficacy, collisions and deadlocks using existing PCA projection
cmap_dict = {"efficacy" : mpl.colormaps["Greens"], "collisions" : mpl.colormaps["Reds"], "deadlocks" : mpl.colormaps["Blues"]}
performance_dict ={"efficacy" : efficacy, "collisions" : collisions, "deadlocks" : deadlocks}
fig, ax = plt.subplots(ncols=3, figsize=(15,4))
for i, title in enumerate(performance_dict.keys()):
    performance = performance_dict[title]
    max_performance = np.max([performance[behaviour] for behaviour in behaviour_list])
    cmap = cmap_dict[title]
    for behaviour in behaviour_list:
        colors = [cmap(p) for p in np.array(performance[behaviour])/max_performance]
        ax[i].scatter(Y_dict[behaviour][:,0], Y_dict[behaviour][:,1], color=colors, label=behaviour)
    # Draw colorbar
    norm = mpl.colors.Normalize(vmin=0, vmax=max_performance)
    mappable = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
    plt.colorbar(mappable=mappable, ax=ax[i])
    ax[i].set_title(title, fontsize=20)
plt.tight_layout()

In [ ]:
# Plot efficacy, collisions and deadlocks using existing PCA projection
cmap_dict = {"efficacy" : mpl.colormaps["Greens"], "collisions" : mpl.colormaps["Reds"], "deadlocks" : mpl.colormaps["Blues"]}
performance_dict ={"efficacy" : efficacy, "collisions" : collisions, "deadlocks" : deadlocks}
fig, ax = plt.subplots(ncols=3, figsize=(15,4))
for i, title in enumerate(performance_dict.keys()):
    performance = performance_dict[title]
    max_performance = np.max([performance[behaviour] for behaviour in behaviour_list])
    cmap = cmap_dict[title]
    for behaviour in behaviour_list:
        colors = [cmap(p) for p in np.array(performance[behaviour])/max_performance]
        ax[i].scatter(embedding_dict[behaviour][:,0], embedding_dict[behaviour][:,1], color=colors, label=behaviour)
    # Draw colorbar
    norm = mpl.colors.Normalize(vmin=0, vmax=max_performance)
    mappable = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
    plt.colorbar(mappable=mappable, ax=ax[i])
    ax[i].set_title(title, fontsize=20)
plt.tight_layout()